# Notebook to start overviewing the data from ECG channel

## Import libraries

In [136]:
import os
import mne
import matplotlib.pyplot as plt
import neurokit2 as nk
import pandas as pd

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 6)
mpl.rcParams['figure.dpi'] = 120

%matplotlib tk

# Part 1: Data Loading

## Load ECG Signal from Parquet

In [137]:
# Constants
sampling_rate = 1000
data_folder = './rest_data_v2'
subject_id = "080"

# Load single subject data for exploration
parquet_file = f"sub-{subject_id}_task-rest_ecg_rest.parquet"
data = pd.read_parquet(os.path.join(data_folder, parquet_file))
# ...existing code...
ecg_signal = data['ecg_nk']
raw_signal = data['ecg_raw']

print(f"✅ Data loaded successfully")
print(f"   Shape: {data.shape}")
print(f"   Columns: {list(data.columns)}")

✅ Data loaded successfully
   Shape: (244661, 4)
   Columns: ['time', 'ecg_raw', 'ecg_mod', 'ecg_nk']


# Part 2: Analysis & Metrics Extraction

## Process ECG Signal

Clean the signal and extract R-peaks for wave delineation

In [138]:
# Process the ECG signal
signals, info = nk.ecg_process(ecg_signal, sampling_rate=sampling_rate)

print(f"✅ ECG processing complete")
print(f"   Signals shape: {signals.shape}")
print(f"   Available metrics: {list(info.keys())}")

✅ ECG processing complete
   Signals shape: (244661, 19)
   Available metrics: ['method_peaks', 'method_fixpeaks', 'ECG_R_Peaks', 'ECG_R_Peaks_Uncorrected', 'ECG_fixpeaks_ectopic', 'ECG_fixpeaks_missed', 'ECG_fixpeaks_extra', 'ECG_fixpeaks_longshort', 'ECG_fixpeaks_method', 'ECG_fixpeaks_rr', 'ECG_fixpeaks_drrs', 'ECG_fixpeaks_mrrs', 'ECG_fixpeaks_s12', 'ECG_fixpeaks_s22', 'ECG_fixpeaks_c1', 'ECG_fixpeaks_c2', 'sampling_rate', 'ECG_P_Peaks', 'ECG_P_Onsets', 'ECG_P_Offsets', 'ECG_Q_Peaks', 'ECG_R_Onsets', 'ECG_R_Offsets', 'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_T_Onsets', 'ECG_T_Offsets']


In [139]:
# Extract R-peaks and clean signal for delineation
rpeaks = info['ECG_R_Peaks']
clean = nk.ecg_clean(ecg_signal, sampling_rate=sampling_rate)

print(f"✅ R-peaks extracted and signal cleaned")
print(f"   Number of R-peaks: {len(rpeaks)}")
print(f"   Clean signal shape: {clean.shape}")

✅ R-peaks extracted and signal cleaned
   Number of R-peaks: 286
   Clean signal shape: (244661,)


In [151]:
# Perform wave delineation to identify P, QRS, and T waves
signals_delineate, waves = nk.ecg_delineate(clean, rpeaks=rpeaks, sampling_rate=sampling_rate, method="cwt", show=True)

print(f"✅ Wave delineation complete")
print(f"   P-waves: {len(waves['ECG_P_Peaks'])}")
print(f"   Q-waves: {len(waves['ECG_Q_Peaks'])}")
print(f"   S-waves: {len(waves['ECG_S_Peaks'])}")
print(f"   T-waves: {len(waves['ECG_T_Peaks'])}")

✅ Wave delineation complete
   P-waves: 285
   Q-waves: 286
   S-waves: 286
   T-waves: 285


## Calculate Heart Rate Variability (HRV)

Compute time, frequency, and non-linear domain HRV indices

In [147]:
# Calculate HRV metrics (Time, Frequency, and Non-Linear domains)
hrv_indices = nk.hrv(info, sampling_rate=sampling_rate, show=True)

print(f"✅ HRV analysis complete")
print(f"   Computed {len(hrv_indices.columns)} HRV indices")
display(hrv_indices)

✅ HRV analysis complete
   Computed 91 HRV indices


,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,853.722807,61.231985,23.572711,58.429993,NaN,NaN,NaN,NaN,68.626515,68.747656,...,1.579301,7.183915,1.491253,1.289951,1.505028,2.207548,1.851117,1.943011,4.742175,0.829789


# Part 3: Visualization & Results

## ECG Signal with Delineated Waves

Visualize processed ECG with P, QRS, and T waves identified

In [146]:
nk.ecg_plot(signals, info)


In [143]:
nk.ecg_peaks(clean, sampling_rate = 1000, show = True)

(        ECG_R_Peaks
 0                 0
 1                 0
 2                 0
 3                 0
 4                 0
 ...             ...
 244656            0
 244657            0
 244658            0
 244659            0
 244660            0
 
 [244661 rows x 1 columns],
 {'method_peaks': 'neurokit',
  'method_fixpeaks': 'None',
  'ECG_R_Peaks': array([   641,   1459,   2303,   3059,   3854,   4686,   5552,   6377,
           7199,   8040,   8915,   9777,  10590,  11451,  12337,  13176,
          14019,  14883,  15761,  16505,  17200,  17886,  18606,  19372,
          20175,  20993,  21800,  22628,  23431,  24263,  25019,  25781,
          26622,  27394,  28251,  29118,  29982,  30789,  31660,  32548,
          33398,  34272,  35162,  36084,  37008,  37916,  38788,  39637,
          40523,  41427,  42302,  43107,  43973,  44878,  45780,  46569,
          47413,  48288,  49197,  50005,  50809,  51666,  52544,  53431,
          54199,  55037,  55944,  56874,  57659,  58476,  59

## Segment

In [144]:
cqrs_epochs = nk.ecg_segment(clean, rpeaks=rpeaks, sampling_rate=1000, show=True)

## Summary Statistics

Key metrics and diagnostic information

In [145]:
# Display comprehensive summary of analysis
print("\n" + "="*70)
print("📊 ECG ANALYSIS SUMMARY")
print("="*70)

print(f"\n📈 Recording Details:")
print(f"   Duration: {len(ecg_signal)/sampling_rate:.2f} seconds")
print(f"   Sampling Rate: {sampling_rate} Hz")
print(f"   Total Samples: {len(ecg_signal):,}")

print(f"\n❤️  Heart Rate Analysis:")
print(f"   R-peaks Detected: {len(info['ECG_R_Peaks'])}")
print(f"   Average HR: {signals['ECG_Rate'].mean():.1f} bpm")
print(f"   HR Min: {signals['ECG_Rate'].min():.1f} bpm")
print(f"   HR Max: {signals['ECG_Rate'].max():.1f} bpm")

print(f"\n🌊 Wave Delineation:")
print(f"   P-waves: {len(info['ECG_P_Peaks'])} detected")
print(f"   Q-waves: {len(info['ECG_Q_Peaks'])} detected")
print(f"   S-waves: {len(info['ECG_S_Peaks'])} detected")
print(f"   T-waves: {len(info['ECG_T_Peaks'])} detected")

print(f"\n📋 HRV Indices Calculated:")
print(f"   Total: {len(hrv_indices.columns)} indices")
print(f"\n   Top HRV Metrics:")
for col in list(hrv_indices.columns)[:5]:
    print(f"   - {col}: {hrv_indices[col].values[0]:.4f}")

print("\n" + "="*70)


📊 ECG ANALYSIS SUMMARY

📈 Recording Details:
   Duration: 244.66 seconds
   Sampling Rate: 1000 Hz
   Total Samples: 244,661

❤️  Heart Rate Analysis:
   R-peaks Detected: 286
   Average HR: 70.3 bpm
   HR Min: 59.9 bpm
   HR Max: 87.5 bpm

🌊 Wave Delineation:
   P-waves: 286 detected
   Q-waves: 286 detected
   S-waves: 286 detected
   T-waves: 286 detected

📋 HRV Indices Calculated:
   Total: 91 indices

   Top HRV Metrics:
   - HRV_MeanNN: 853.7228
   - HRV_SDNN: 61.2320
   - HRV_SDANN1: 23.5727
   - HRV_SDNNI1: 58.4300
   - HRV_SDANN2: nan

